In [ ]:
#!pip install accelerate -U
#!pip install --upgrade transformers==4.52.3

In [ ]:
import transformers
print(transformers.__version__)


4.52.3


In [ ]:
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import pandas as pd
from datasets import Dataset
import numpy as np

prepare data

In [ ]:
# Load and prepare dataset
df = pd.read_parquet('/content/drive/MyDrive/NLP News Bias Data/data_newsbias_cleaned.parquet')
df = df.dropna(subset=['text', 'bias_label']).copy()
df['bias_label'] = df['bias_label'].astype(str)

df = (
    df.groupby('bias_label', group_keys=False)
      .apply(lambda g: g.sample(50_000, random_state=42))
      .reset_index(drop=True)
)

# Label encoding
label2id = {label: idx for idx, label in enumerate(sorted(df['bias_label'].unique()))}
id2label = {v: k for k, v in label2id.items()}
df['label'] = df['bias_label'].map(label2id)

# Split
train_df, test_df = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)

<ipython-input-5-eab181ba4394>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(50_000, random_state=42))


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")  # ← use "bert-base-uncased"

def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding=False,
        max_length=256
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Convert to Hugging Face Datasets
train_ds = Dataset.from_pandas(train_df[['text', 'label']])
test_ds = Dataset.from_pandas(test_df[['text', 'label']])

train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

# Pad dynamically per batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",   # ← change from "roberta-base"
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


build the model

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert-base-uncased",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, preds)}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



<ipython-input-10-cedb8011fcec>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


train

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.370700,0.426431,0.852867


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.370700,0.426431,0.852867
2,0.208700,0.564733,0.859767
3,0.132800,0.727311,0.862567


TrainOutput(global_step=45000, training_loss=0.28362311380174426, metrics={'train_runtime': 9352.0332, 'train_samples_per_second': 38.494, 'train_steps_per_second': 4.812, 'total_flos': 4.736041519104e+16, 'train_loss': 0.28362311380174426, 'epoch': 3.0})

test

In [ ]:
import numpy as np
preds = trainer.predict(test_ds)
print("\nFinal Accuracy:", accuracy_score(preds.label_ids, np.argmax(preds.predictions, axis=1)))
print("\nClassification Report:\n", classification_report(preds.label_ids, np.argmax(preds.predictions, axis=1), target_names=list(label2id.keys())))


🎯 Final Accuracy: 0.8625666666666667

📋 Classification Report:
               precision    recall  f1-score   support

      center       0.88      0.86      0.87     10000
        left       0.83      0.89      0.86     10000
       right       0.88      0.85      0.86     10000

    accuracy                           0.86     30000
   macro avg       0.86      0.86      0.86     30000
weighted avg       0.86      0.86      0.86     30000



evaluate

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix



y_true = preds.label_ids
y_pred = np.argmax(preds.predictions, axis=1)

# 2) Confusion matrix
cm      = confusion_matrix(y_true, y_pred)
cm_norm = cm.astype(float) / cm.max()  # normalize 0–1 for thresholding

# 3) Plot
fig, ax = plt.subplots(figsize=(6,5))
cmap = plt.cm.viridis
im = ax.imshow(cm, cmap=cmap)

# ticks
labels = list(label2id.keys())
ax.set_xticks(np.arange(len(labels)))
ax.set_yticks(np.arange(len(labels)))
ax.set_xticklabels(labels, rotation=45, ha="right")
ax.set_yticklabels(labels)
ax.set_xlabel("Predicted")
ax.set_ylabel("True")
ax.set_title("Confusion Matrix")

# 4) Annotate with dynamic text color
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        val = cm[i, j]
        # normalized to [0–1] for that cell
        norm_val = cm_norm[i, j]
        # if background is dark (norm_val < 0.5) use white text, else black
        color = "white" if norm_val < 0.5 else "black"
        ax.text(j, i, f"{val}", ha="center", va="center", color=color)

fig.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()